In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('result_project_query_all_flights.csv')

In [4]:
data

,flight_id,arrival_airport,actual_departure,actual_arrival,aircraft_model,seats_per_craft,fuel_consumption,fuel_price_by_month,flight_income,sold_tickets,flight_cost,pnl,rn,flight_occupancy
0,136642,EGO,2017-01-30 09:28:00+00,2017-01-30 10:17:00+00,Sukhoi Superjet-100,97,1.7,41435,531000.0,64,57526,473474.0,1,65
1,136807,EGO,2017-02-23 09:28:00+00,2017-02-23 10:18:00+00,Sukhoi Superjet-100,97,1.7,39553,531000.0,68,56033,474967.0,2,70
2,136844,EGO,2017-02-28 09:26:00+00,2017-02-28 10:16:00+00,Sukhoi Superjet-100,97,1.7,39553,575100.0,79,56033,519067.0,3,81
3,136887,EGO,2017-01-20 09:30:00+00,2017-01-20 10:19:00+00,Sukhoi Superjet-100,97,1.7,41435,595200.0,78,57526,537674.0,4,80
4,136922,EGO,2017-02-11 09:27:00+00,2017-02-11 10:17:00+00,Sukhoi Superjet-100,97,1.7,39553,607800.0,76,56033,551767.0,5,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,136187,SVO,2016-12-01 10:09:00+00,2016-12-01 11:49:00+00,Boeing 737-300,130,7.8,38867,1873800.0,129,505271,1368529.0,86,99
189,136465,SVO,2016-12-15 10:05:00+00,2016-12-15 11:47:00+00,Boeing 737-300,130,7.8,38867,1886000.0,130,515376,1370624.0,87,100
190,136273,SVO,2016-12-09 10:06:00+00,2016-12-09 11:45:00+00,Boeing 737-300,130,7.8,38867,1873800.0,129,500218,1373582.0,88,99
191,136368,SVO,2016-12-31 10:07:00+00,2016-12-31 11:47:00+00,Boeing 737-300,130,7.8,38867,1886000.0,130,505271,1380729.0,89,100
